<a href="https://colab.research.google.com/github/milanazhang/data-course-sample/blob/main/cf-surprise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# cf surprise

## 基礎建設

In [1]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2022-01-08 15:42:20--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.5’

All_Beauty.csv.5    100%[===================>]  14.78M  17.9MB/s    in 0.8s    

2022-01-08 15:42:21 (17.9 MB/s) - ‘All_Beauty.csv.5’ saved [15499476/15499476]

--2022-01-08 15:42:21--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.5’

meta_All_Beauty.jso 100%[===================>]   9.85M  14.2MB/s    in 0.7s   

In [3]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [4]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [5]:
ratings.head()

,asin,reviewerID,overall,unixReviewTime
0,0143026860,A1V6B6TNIC10QE,1.0,1424304000
1,0143026860,A2F5GHSXFQ0W6J,4.0,1418860800
2,0143026860,A1572GUYS7DGSR,4.0,1407628800
3,0143026860,A1PSGLFK1NSVO,5.0,1362960000
4,0143026860,A6IKXKZMTKGSC,5.0,1324771200


## 資料整理

In [6]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

## EDA

In [86]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [80]:
from datetime import timedelta
ratings.drop_duplicates(inplace=True)
date_sort = ratings.sort_values(['DATE'],ascending=True).groupby(["reviewerID","DATE"]).head(25)
date_sort['dur'] = date_sort.groupby('reviewerID')["DATE"].apply(lambda x: x.diff(1))
repurchase = date_sort[date_sort["dur"].notna()][date_sort[date_sort["dur"].notna()]["dur"] != "0 days"]
repurchase["dur"] = repurchase["dur"].astype("str").str.extract(r'(\d+) (.*$)')[0].astype(int)
purchase_dur = {d["reviewerID"]:d["dur"] for d in repurchase.groupby("reviewerID").agg({"dur":pd.Series.median}).reset_index()[["reviewerID","dur"]].to_dict('records')}

In [123]:
# 使用者大多間隔一年半左右上去一次平台買東西
fig = px.box(repurchase, x = "dur", orientation='h')
fig.show()

In [124]:
ratings["year"] = ratings["DATE"].astype(str).str.extract("(\d\d\d\d)-(\d\d)-(\d\d)")[0]
ratings["month"] = ratings["DATE"].astype(str).str.extract("(\d\d\d\d)-(\d\d)-(\d\d)")[1]
ratings["day"] = ratings["DATE"].astype(str).str.extract("(\d\d\d\d)-(\d\d)-(\d\d)")[2]

In [129]:
ratings_year = ratings[ratings["DATE"] > "2017-09-01"]
ratings_month_dict = {d["month"]:d["reviewerID"] for d in ratings_year[["month","reviewerID"]].groupby("month",as_index=False).agg(set).to_dict("record")}

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:1490: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.



In [143]:
old_cos = set(ratings[ratings["DATE"] < "2017-09-01"]["reviewerID"].to_list())
overlap_cos = {}
new_cos = {}
for m,i in ratings_month_dict.items():
  overlap_cos[m] = len(i&old_cos)/len(i)
  new_cos[m] = len(i-old_cos)/len(i)
  old_cos = old_cos.union(i)

In [144]:
overlap_cos

{'01': 0.04276810659789383,
 '02': 0.047585394581861014,
 '03': 0.04575441583315599,
 '04': 0.04494902687673772,
 '05': 0.05412156536219817,
 '06': 0.05404486743711761,
 '07': 0.049723756906077346,
 '08': 0.040615384615384616,
 '09': 0.11006233303650935,
 '10': 0.08358005010248235,
 '11': 0.06964553170244633,
 '12': 0.06031077166183829}

In [145]:
new_cos

{'01': 0.9572318934021061,
 '02': 0.9524146054181389,
 '03': 0.954245584166844,
 '04': 0.9550509731232623,
 '05': 0.9458784346378019,
 '06': 0.9459551325628824,
 '07': 0.9502762430939227,
 '08': 0.9593846153846154,
 '09': 0.8899376669634906,
 '10': 0.9164199498975176,
 '11': 0.9303544682975536,
 '12': 0.9396892283381617}

In [146]:
overlap_cos = {}
new_cos = {}
last_m = {}
for m,i in ratings_month_dict.items():
  if m == "01":
    last_m = i
    continue
  else:
    overlap_cos[m] = len(i&last_m)/len(i)
    new_cos[m] = len(i-last_m)/len(i)
  last_m = i

In [147]:
overlap_cos

{'02': 0.003297997644287397,
 '03': 0.0023409236007661204,
 '04': 0.0023169601482854493,
 '05': 0.0038856508465167916,
 '06': 0.002379333786539769,
 '07': 0.0016999575010624734,
 '08': 0.0018461538461538461,
 '09': 0.0008904719501335708,
 '10': 0.012525620587565475,
 '11': 0.0027458811782326512,
 '12': 0.0018435607058203845}

In [148]:
new_cos

{'02': 0.9967020023557126,
 '03': 0.9976590763992339,
 '04': 0.9976830398517146,
 '05': 0.9961143491534832,
 '06': 0.9976206662134602,
 '07': 0.9983000424989376,
 '08': 0.9981538461538462,
 '09': 0.9991095280498664,
 '10': 0.9874743794124345,
 '11': 0.9972541188217674,
 '12': 0.9981564392941796}

## 資料切分

In [116]:
#具上述EDA結果，決定切分一年半作為training data
ratings_trainings = ratings[
    (ratings['DATE'] > '2017-03-01') &
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

## 產生推薦

In [117]:
!pip install surprise

In [118]:
import time
import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline

In [119]:
def recommender(training_data, users=[], k=10, user_based=False, algo=KNNBasic, method = "cosine"):

    training_data = (
        training_data
        .sort_values("DATE", ascending=False)
        .groupby(['reviewerID', 'asin']).head(1)
    )

    reader = Reader(rating_scale=(0, 5))
    training_data = training_data[['reviewerID', 'asin', 'overall']]
    data = Dataset.load_from_df(training_data, reader=reader)

    sim_options = {
        'name': method,
        'user_based': user_based  # compute similarities between items
    }
    algo_impl = algo(sim_options=sim_options)
    trainset = data.build_full_trainset()
    algo_impl.fit(trainset)

    recommendation = {}
    for user in users:
        items_user_rated = set(training_data.loc[training_data['reviewerID'] == user]['asin'].to_list())
        recommend_item_list = []
        recommend_item_set = set()
        for item in items_user_rated:
            iid = algo_impl.trainset.to_inner_iid(item)
            recommend_items_iid = algo_impl.get_neighbors(iid, k)
            for sim_item_iid in recommend_items_iid:
                item_raw_id = algo_impl.trainset.to_raw_iid(sim_item_iid)
                if item_raw_id not in items_user_rated and item_raw_id not in recommend_item_set:
                    recommend_item_list.append(item_raw_id)
                    recommend_item_set.add(item_raw_id)

            if len(recommend_item_list) >= k:
                recommend_item_list = recommend_item_list[:k]
                break
        recommendation[user] = recommend_item_list

    return recommendation


ratings_by_user = recommender(ratings_trainings, users, k=10, user_based=False, algo=KNNBasic, method = "cosine")
ratings_by_user2 = recommender(ratings_trainings, users, k=10, user_based=False, algo=KNNBasic, method = "pearson")
ratings_by_user3 = recommender(ratings_trainings, users, k=10, user_based=False, algo=KNNBasic, method = "pearson_baseline")
ratings_by_user4 = recommender(ratings_trainings, users, k=10, user_based=False, algo=KNNBasic, method = "msd")

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


## 結果評估

In [120]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    recommended_user = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))
            if len(ratings_by_user[d]) != 0:
              recommended_user += 1

    score = total / len(ratings_testings)
    
    return score,total,recommended_user


In [122]:
print("method || (recall_score, predict_right, recommended_user)")
for m,r in list(zip(["cosine","pearson","pearson_baseline","msd"],[ratings_by_user,ratings_by_user2,ratings_by_user3,ratings_by_user4])):
  print(m,evaluate(ratings_testings_by_user, r),sep=" || ")

method || (recall_score, predict_right, recommended_user)
cosine || (0.001694915254237288, 1, 19)
pearson || (0.003389830508474576, 2, 19)
pearson_baseline || (0.003389830508474576, 2, 19)
msd || (0.001694915254237288, 1, 19)


## 總結

* 使用super套件，在cosine similarity算法上，結果與前面使用手刻程式差不多。但皮爾森相似度，預測中的商品推薦數多一個
* 演算法使用過KNNBasic與其他KNN算法，結果都一樣，而KNN以外演算法，因為是屬於數值型的預測，無法產出分群結果，所以都沒有使用
* 消費者大都集中在一年半內上一次平台購買，故測試集切在一年半，比較能找到購買兩種商品以上的消費者，提高找到相似產品的機率
* 平台每個月的使用者，只有少部分是過去有買過的，其他都是新使用者，故使用歷史資料預測下個月的消費者偏好，是不太適合的作法，這也是collaberative filtering推薦效果差的原因